In [14]:
import qiskit
import numpy as np

import json, glob, math

In [8]:
glob.glob("*")

['curves.nb', 'elephant QFT.ipynb', 'README.md']

In [39]:
def qubits_necessary(n):
    """log2 of n"""
    l2 = math.log2(n)
    assert l2 % 1 == 0, "not an integer power of 2"
    return int(l2)

def normalized(arr):
    """l2-normalization"""
    return arr / np.linalg.norm(arr, 2)

## Data Loading

In [54]:
with open("./Resources/elephant.3.json") as f:
    data = json.load(f)[:-1]
    data = normalized(np.array([complex(a, b) for a, b in data]))
    data = np.round(data, 2)

In [55]:
data

array([-0.34-0.2j , -0.35+0.03j, -0.37+0.21j,  0.25+0.2j ,  0.25+0.34j,
        0.31+0.04j,  0.2 -0.23j, -0.07-0.25j])

## Classical FT

In [68]:
data_ft = normalized(np.fft.fft(data))

In [69]:
data_ft

array([-0.04245614+0.04953216j, -0.18816477-0.01940699j,
        0.07076023+0.13444443j, -0.00916641-0.19257912j,
       -0.14152046+0.03538011j,  0.08202443+0.04063506j,
       -0.01415205-0.02122807j, -0.71966393-0.59285941j])

## State Loading

In [92]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute

In [93]:
def to_statevector(self):
    job = execute(self, Aer.get_backend("statevector_simulator"))
    return job.result().get_statevector()

QuantumCircuit.to_statevector = to_statevector

In [108]:
# number of qubits necessary
qubit_count = qubits_necessary(len(data_ft))
print(f"need {qubit_count} qubits")

# quantum and classical registers
quantum_registers = QuantumRegister(qubit_count)
classical_registers = ClassicalRegister(qubit_count)

# array of complex amplitudes
desired_state = data_ft

# initialization circut
circuit_init = QuantumCircuit(quantum_registers, classical_registers, name="data_loader_trivial")
circuit_init.initialize(desired_state, range(qubit_count))

need 3 qubits


In [109]:
circuit_init.to_statevector()

array([-0.04245614+0.04953216j, -0.18816477-0.01940699j,
        0.07076023+0.13444443j, -0.00916641-0.19257912j,
       -0.14152046+0.03538011j,  0.08202443+0.04063506j,
       -0.01415205-0.02122807j, -0.71966393-0.59285941j])

## Inverse QFT

In [110]:
def QHomer_IQFT(qreg, circuit):
    from qiskit.aqua.components.iqfts.standard import Standard as StandardIQFT

    IQFT = StandardIQFT(len(quantum_registers))
    iqft = IQFT.construct_circuit(mode="circuit", qubits=qreg, circuit=circuit)

    return iqft


In [113]:
circuit_iqft = QHomer_IQFT(quantum_registers, circuit_init)

## Simulate

In [114]:
simulator = Aer.get_backend("qasm_simulator")
result = execute(circuit_iqft, backend = simulator, shots = 1).results()


No measurements in circuit "data_loader_trivial": count data will return all zeros.


AttributeError: 'AerJob' object has no attribute 'results'